# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 2)

# Table of Contents
- [Introduction: Business Problem](#introduction)
- [Data](#data)
- [Methodology](#methodology)
- [Results](#results)
- [Discussion](#discussion)
- [Conclusion](#conclusion)

### Introduction - Problem Description <a name="introduction"></a>
"In a city of your choice, if someone is looking open their own business, where would you recommend that they setup their office?"


### Data - Description <a name="data"></a>
Toronto is the capital city Canada with a population of over 2.93 million people. Choosing a neighborhood to live in or open a business is a difficult task that requires informed decision based on location data. The extracted data contains information such as Postal Code, Borough, and Neighborhood. Combined with coordinate data e.g. latitude, longitude of each neighborhood, we can abstract essential details on the most common or popular business in the local districts.


### Methodology <a name="methodology"></a>
- We explored, segmented, and clustered the neighborhoods in the city of Toronto based on the **postal code** and **borough** information.

For the Toronto neighborhood data, a **Wikipedia** page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. The data from the Wikipedia page was scraped and wrangled, cleaned, and then read it into a pandas  dataframe so that it is in a structured format. Additionally, the location data (latitude, longitude) was obtained from each neighborhood using **Foursquare** - a widely accessible Application Program Interface (API) tool to obtain coordinates of neighborhoods. 

Specifically, after retreiving the URL and creating a Beautiful soup object, a list was created. Next, after finding the table and table data, a dictionary was created having 3 keys: PostalCode, Borough and Neighborhood Split, strip and replace functions were utilized to obtain Borough and Neighborhood information. The list was appended and a dataframe was created with the list. Once the data was in a structured format, analysis was performed to explore and cluster the neighborhoods in the city of Toronto.

Visualization of the data points were generated using the `Folium` library. Further analysis was performed using **one-hot encoding** to categorize the surrounding businesses e.g. yoga studio, airport, restaurants etc. A list of the 10 most common venues was generated by grouping rows and taking the mean of the frequency of each category. Cluster analysis was performed using K-means method to generate five distinct groups. A new dataframe was generated combining the clusters and top ten venues for each neighborhood.

The approach to this problem will include the following: 
- Scrape the Wikipedia page
- Wrangle the data, clean it
- Read it into a pandas dataframe
- Explore and cluster the neighborhoods


### Results <a name="results"></a>

The results from the cluster analysis indicate 5 distinct clusters in the Toronto area (Tables 1 - 5). In cluster 1, the top 10 most common venues can be found in the following Boroughs: East Toronto and Central Toronto. In East Toronto, the top most common venues include: Pub, Greek Restaurant, Park, Coffee Shop. In Central Toronto, the most common venues include: Gym/Fitness Center, Clothing Store, Pizza Place, and Coffee Shop. In cluster 2, 3, and 4 a Park is the top most common venue located in Central Toronto, East York/East Toronto, and Downtown Toronto respectively. In cluster 5, the most common venue was a Garden located in Central Toronto.

### Discussion <a name="discussion"></a>

- Discuss any observations you noted and any recommendations you can make based on the results

In summary, the top boroughs in the Toronto area include East Toronto, Central Toronto, Downtown Toronto, and East/York/East Toronto. The 1st most common businesses are food related e.g. Pub (East Toronto), Greek (East Toronto), Coffee (East and Central Toronto), Pizza Place (Central Toronto). Another popular set of businesses are health and department store-related e.g. Gym/Fitness Center(Central Toronto), Clothing Store(Central Toronto). It is worth highlighting that coffee shops appear to be a popular business and is the most common venue in both East Toronto and Central Toronto. This may be an appealing option for coffee shops eager to expand their business front in a growing market. However, one should take into consideration that these data was obtained from https://www.wikipedia.org/ where any user can create, delete or edit articles.

Additionally, a popular attraction in the Toronto area includes outdoor activities such as Garden (Central Toronto) and Parks in Central, East York/East Toronto, and Downtown Toronto. This descriptive information may provide a window into the outdoor and active lifestyle of the local community. Again, any conclusions drawn from these data are based on information pulled to date (April 6th, 2021) and are subject to change in the context of the current climate e.g. status of COVID-19 pandemic, seasonal environmental changes etc. As many indoor activities are limited to capacity requirements (25% - 50%) depending on the number of new cases of COVID-positive individuals, it makes sense that outdoor activities i.e. parks and garden are a popular option for socializing.

### Conclusion <a name="conclusion"></a>

Based on neighborhood and location data, coffee shops appeared twice as a most common venue both East Toronto, and Central Toronto. Someone looking to open their business can expect to compete with these businesses but may also have a strong opportunity to access a growing customer base. It may also be in the business's best interest to have favorable outdoor amenities to account for capacity restrictions and necessary health precautions to minimize any potential outbreak related to COVID-19 transmission.

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

#to convert address into latitude and longitude values
from geopy.geocoders import Nominatim 

#transform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

#map rendering library
import folium

#import k-means from clustering stage
from sklearn.cluster import KMeans

#matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('libraries imported')

libraries imported


In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'html5lib')

#using provided sample code
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [3]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [4]:
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
print("Shape: ", df.shape)

Shape:  (103, 3)


### Get Latitude and Longitude coordinates of each neighborhood
- Use Foursquare to obtain coordinates of each neighborhood.

In [6]:
df_geo_coor = pd.read_csv('https://cocl.us/Geospatial_data')
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df_toronto = pd.merge(df, df_geo_coor, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Explore and cluster neighborhoods in Toronto
- Get the latitude and longtiude values of Toronto


In [8]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


### Create a map of the whole Toronto City with Neighborhoods superimposed
- add markers

In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [11]:
#adding markers
for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

### Map section of Toronto City
- filter for boroughs that contain **Toronto**
- re-plot map with updated markers for this region

In [12]:
#contains 'Toronto'
df_toronto_denc = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_denc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106
2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
4,M4M,East Toronto,Studio District,43.659526,-79.340923


In [13]:
#replot
map_toronto_denc = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_toronto_denc['Latitude'], 
        df_toronto_denc['Longitude'], 
        df_toronto_denc['Borough'], 
        df_toronto_denc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_denc)  

map_toronto_denc


# Define Foursquare Credential and Version
- insert foursquare credentials from an active developer account: [here](https://foursquare.com/developers/apps)

In [14]:
CLIENT_ID = 'GEFHYB20NWQRWJRE3AR15K3ZPJJSEM3141QK3KCUDCYW1LIU'
CLIENT_SECRET = 'YAXM5JRYI4VJUFHI0ZRY2R0F5ZC2GNPXQHAPFTFNDRETAKAC'
VERSION = '20180604'

### Explore first neighborhood
- Get the neighborhood's latitude and longitude values.

In [15]:
neighborhood_name = df_toronto_denc.loc[0, 'Neighborhood']
print(f"The first neighborhood's name is '{neighborhood_name}'.")

The first neighborhood's name is 'The Beaches'.


In [16]:
#longitude/latitude
neighborhood_latitude = df_toronto_denc.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto_denc.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


### Get the top 100 venues in 'The Beaches' within a 500 meter radius
- extract the category of the venue


In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [19]:
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Seaspray Restaurant,Asian Restaurant,43.678888,-79.298167


## Explore neighborhoods in a part of Toronto City
We are working on the data frame df_toronto_denc. Recall that, this region contain DENC of Toronto where,

First, let's create a function to repeat the same process to all the neighborhoods in DENC of Toronto.

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called **toronto_denc_venues**

In [22]:
toronto_denc_venues = getNearbyVenues(names=df_toronto_denc['Neighborhood'],
                                   latitudes=df_toronto_denc['Latitude'],
                                   longitudes=df_toronto_denc['Longitude']
                                  )

In [23]:
toronto_denc_venues.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


Check how many venues were returned for each neighborhood

In [ ]:
toronto_denc_venues.groupby('Neighborhood').count()

Find out how many unique categories can be curated from all the returned venues

In [24]:
print('There are {} uniques categories.'.format(len(toronto_denc_venues['Venue Category'].unique())))

There are 231 uniques categories.


### Analyze Each Neighborhood

In [25]:
# one hot encoding
toronto_denc_onehot = pd.get_dummies(toronto_denc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_denc_onehot['Neighborhood'] = toronto_denc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_denc_onehot.columns[-1]] + list(toronto_denc_onehot.columns[:-1])
toronto_denc_onehot = toronto_denc_onehot[fixed_columns]

toronto_denc_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
toronto_denc_grouped = toronto_denc_onehot.groupby('Neighborhood').mean().reset_index()
toronto_denc_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.125,0.125,0.125,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,Central Bay Street,0.016129,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.016129,0.0,0.0,0.016129,0.0
4,Christie,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


- Check the 10 most common venues in each neighborhood

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_denc_grouped['Neighborhood']

for ind in np.arange(toronto_denc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_denc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Restaurant,Beer Bar,Seafood Restaurant,Pharmacy,Farmers Market,Cheese Shop,Park
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Stadium,Intersection,Bar,Bakery,Restaurant,Climbing Gym,Furniture / Home Store
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Boat or Ferry,Rental Car Location,Coffee Shop,Plane,Bar,Harbor / Marina
3,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Thai Restaurant,Salad Place,Bubble Tea Shop,Park,Poke Place
4,Christie,Grocery Store,Café,Park,Nightclub,Candy Store,Baby Store,Italian Restaurant,Athletics & Sports,Restaurant,Coffee Shop


### Cluster neighborhoods
- Run k-means to cluster the neighborhood into 5 clusters.

In [29]:
# set number of clusters
kclusters = 5

toronto_denc_grouped_clustering = toronto_denc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_denc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

- Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_denc_merged = df_toronto_denc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_denc_merged = toronto_denc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_denc_merged.head() # check the last columns!




,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Asian Restaurant,Health Food Store,Trail,Wings Joint,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store
1,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106,2,Park,Convenience Store,Intersection,Wings Joint,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Bookstore,Brewery,Indian Restaurant,Spa
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Fast Food Restaurant,Pet Store,Pub,Sandwich Place,Fish & Chips Shop,Italian Restaurant,Steakhouse,Restaurant,Sushi Restaurant
4,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,American Restaurant,Bakery,Brewery,Café,Gastropub,Gym / Fitness Center,Diner,Park,Middle Eastern Restaurant


- Finally, visualize the resulting clusters

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto_denc_merged['Latitude'], 
        toronto_denc_merged['Longitude'], 
        toronto_denc_merged['Neighborhood'], 
        toronto_denc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters
- Examine each cluster and determine the discriminating venue categories that distinguish each cluster.



### Cluster 1

In [37]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 0, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Pub,Asian Restaurant,Health Food Store,Trail,Wings Joint,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store
2,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Bookstore,Brewery,Indian Restaurant,Spa
3,East Toronto,0,Park,Fast Food Restaurant,Pet Store,Pub,Sandwich Place,Fish & Chips Shop,Italian Restaurant,Steakhouse,Restaurant,Sushi Restaurant
4,East Toronto,0,Coffee Shop,American Restaurant,Bakery,Brewery,Café,Gastropub,Gym / Fitness Center,Diner,Park,Middle Eastern Restaurant
6,Central Toronto,0,Gym / Fitness Center,Breakfast Spot,Hotel,Food & Drink Shop,Department Store,Convenience Store,Park,Sandwich Place,Gym,Aquarium
7,Central Toronto,0,Clothing Store,Coffee Shop,Gym / Fitness Center,Sporting Goods Shop,Café,Chinese Restaurant,Diner,Fast Food Restaurant,Gift Shop,Ice Cream Shop
8,Central Toronto,0,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Italian Restaurant,Café,Gym,Sushi Restaurant,Seafood Restaurant,Farmers Market
10,Central Toronto,0,Coffee Shop,Pub,Liquor Store,Restaurant,Bank,Bagel Shop,Supermarket,Sushi Restaurant,Light Rail Station,Fried Chicken Joint
12,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Pizza Place,Park,Pub,Italian Restaurant,Bakery,Butcher,Caribbean Restaurant
13,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Yoga Studio,Mediterranean Restaurant,Fast Food Restaurant,Hotel,Gay Bar,Men's Store


### Cluster 2

In [38]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 1, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Central Toronto,1,Park,Tennis Court,Restaurant,Wings Joint,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


### Cluster 3

In [39]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 2, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East York/East Toronto,2,Park,Convenience Store,Intersection,Wings Joint,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
11,Downtown Toronto,2,Park,Playground,Trail,Wings Joint,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


### Cluster 4

In [40]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 3, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,3,Park,Bus Line,Swim School,Wings Joint,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
24,Central Toronto,3,Park,Trail,Jewelry Store,Bus Line,Sushi Restaurant,Wings Joint,Distribution Center,Event Space,Ethiopian Restaurant,Escape Room


### Cluster 5

In [41]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 4, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,4,Garden,Home Service,Ice Cream Shop,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
